In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import torch

# Aumentación de datos con `torchvision`

Si tenemos un dataset de imágenes muy pequeño nuestro modelo podría sobreajustarse

Podemos intentar incrementar nuestra dataset usando **transformaciones**

Si rotamos, trasladamos o cambiamos el brillo de una imagen obtendremos una nueva imagen "casi siempre" de la misma clase

*torchvision* tiene funciones implementadas para hacer transformaciones en imágenes

- [Rotación aleatoria](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.RandomRotation)
- [Espejamiento aleatorio](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.RandomHorizontalFlip)
- [Cropping aleatorio](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.RandomCrop): Recortar la imagen
- [Cambios aleatorios de brillo y contraste](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.ColorJitter)
- [Transformación afin aleatoria](https://pytorch.org/docs/stable/torchvision/transforms.html)
- [entre otros](https://pytorch.org/docs/stable/torchvision/transforms.html)

Cada transformación permite especificar límites, por ejemplo "máximo ángulo de rotación", "máxima distorsión de brillo", etc

Las transformaciones también sirven para hacer que la red gane "invarianzas"

**Ejemplo:** Si entrenamos con copias rotadas de nuestras imágenes, la red se volverá invariante a la rotación

## ATENCIÓN

> Las transformaciones que apliquemos no deben cambiar la interpretación de clase

- Si rotas un seis en 180 grados se convierte en un nueve
- Si cambias demasiado el tono (hue) podrías obtener colores distintos a la realidad (¿perro verde?)



# Transformaciones aleatorias

La mayoría de las transformaciones están diseñadas para aplicarse sobre imágenes en formato PIL

Podemos componer varias transformaciones usando [`torchvision.transforms.Compose`](https://pytorch.org/docs/stable/torchvision/transforms.html)

In [ ]:
from torchvision import transforms

from PIL import Image

img = Image.open("img/dog.jpg")

my_transform = transforms.Compose([transforms.Resize(200),
                                   transforms.RandomHorizontalFlip(),
                                   transforms.RandomRotation(degrees=30),
                                   transforms.ColorJitter(brightness=0.5, contrast=0.5,
                                                          saturation=0.5, hue=0.0),
                                  ])

display(transforms.Resize(200)(img), my_transform(img))

# Entrenando con datos aumentados

Podemos componer una transformación y añadirla a un dataset

Luego cuando usamos el dataloader se generaran imágenes con transformaciones aleatorias

> **NUNCA OLVIDES** SOLO SE AUMENTA EL CONJUNTO DE ENTRENAMIENTO


In [ ]:
from torchvision import datasets
from torch.utils.data import DataLoader

mnist_transform = transforms.Compose([transforms.RandomAffine(degrees=30, translate=(0.2, 0.2), 
                                                              scale=(0.5, 1.5), shear=None, 
                                                              resample=False, fillcolor=0),
                                      transforms.ColorJitter(brightness=0.5, contrast=0.5, 
                                                             saturation=0.5, hue=0.0),
                                      transforms.ToTensor()
                                     ])
from torchvision import datasets
from torch.utils.data import DataLoader


mnist_train_data = datasets.MNIST(root='~/datasets',
                                  train=True, download=True,
                                  transform=mnist_transform)

train_loader = DataLoader(mnist_train_data, shuffle=False, batch_size=32)

for image, label in train_loader:
    break

fig, ax = plt.subplots(4, 8, figsize=(7, 4), tight_layout=True)
for k in range(32):
    i, j = np.unravel_index(k, (4, 8))
    ax[i, j].axis('off')
    ax[i, j].set_title(label[k].numpy())
    ax[i, j].imshow(image[k].numpy()[0, :, :], cmap=plt.cm.Greys_r)